In [1]:
import json

def load_jsonl(file_path):
    """加载 jsonl 文件"""
    with open(file_path, "r", encoding="utf-8") as f:
        return [json.loads(line.strip()) for line in f if line.strip()]

def save_jsonl(file_path, data):
    """保存 jsonl 文件"""
    with open(file_path, "w", encoding="utf-8") as f:
        for entry in data:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")

def merge_and_dedup_jsonl(files, output_path):
    all_entries = []
    seen_ids = set()

    for file in files:
        data = load_jsonl(file)
        for entry in data:
            entry_id = entry.get("id")
            if entry_id and entry_id not in seen_ids:
                seen_ids.add(entry_id)
                all_entries.append(entry)

    print(f"合并后总条数: {len(all_entries)}")
    save_jsonl(output_path, all_entries)
    print(f"✅ 已保存至: {output_path}")

if __name__ == "__main__":
    input_files = [
        "/mnt/sda/wyp/forestllm-main/output/step5/part1_step5.jsonl",
        "/mnt/sda/wyp/forestllm-main/output/step5/part2_step5.jsonl",
        "/mnt/sda/wyp/forestllm-main/output/step5/part3_step5.jsonl"
    ]
    output_file = "/mnt/sda/wyp/forestllm-main/output/step5/merged_step5_dedup.jsonl"
    merge_and_dedup_jsonl(input_files, output_file)


合并后总条数: 5718
✅ 已保存至: /mnt/sda/wyp/forestllm-main/output/step5/merged_step5_dedup.jsonl


In [1]:
import os
import math

def split_jsonl_three_parts(file_path, output_dir=None):
    if output_dir is None:
        output_dir = os.path.dirname(file_path)

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    total = len(lines)
    part_size = math.ceil(total / 3)

    parts = [
        lines[:part_size],
        lines[part_size:part_size * 2],
        lines[part_size * 2:]
    ]

    for i, part in enumerate(parts, 1):
        part_path = os.path.join(output_dir, f"part{i}.jsonl")
        with open(part_path, "w", encoding="utf-8") as f:
            f.writelines(part)
        print(f"📄 Part {i}: {part_path} ({len(part)} lines)")

    print(f"✅ Split completed: {total} lines into 3 parts.")

# 调用
split_jsonl_three_parts("/mnt/sda/wyp/forestllm-main/output/step5/merged_step4_dedup.jsonl", "/mnt/sda/wyp/forestllm-main/output/step5")




📄 Part 1: /mnt/sda/wyp/forestllm-main/output/step5/part1.jsonl (1906 lines)
📄 Part 2: /mnt/sda/wyp/forestllm-main/output/step5/part2.jsonl (1906 lines)
📄 Part 3: /mnt/sda/wyp/forestllm-main/output/step5/part3.jsonl (1906 lines)
✅ Split completed: 5718 lines into 3 parts.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 定义模型数据
models = [
    "Faster R-CNN", "Cascade R-CNN", "YOLOV5n", "YOLOV8n", "YOLOV10n", "YOLOV11n", 
    "YOLOV5s", "YOLOV8s", "YOLOV10s", "YOLOV11s", "RT-DETR", "RT-DETRv2", "DINO", "MSAR-T", "MSAR-B"
]

gflops = [91.3, 119.0, 7.1, 8.1, 8.4, 6.4, 23.8, 28.4, 24.4, 21.6, 130.5, 100.5, 119.0, 12.2, 47.8]
map_values = [51.7, 47.9, 67.2, 74.2, 63.4, 68.8, 68.9, 79.9, 73.4, 76.1, 53.8, 64.0, 57.0, 76.3, 81.9]
params = [41.7, 69.4, 2.5, 3.0, 2.7, 2.6, 9.1, 11.1, 8.0, 9.4, 42.7, 36.4, 47.5, 5.5, 20.7]

# 归一化气泡大小
bubble_size = np.array(params) * 10

# 颜色区分不同算法
colors = [
    'blue', 'blue', 'green', 'green', 'green', 'green', 'orange', 'orange', 'orange', 'orange', 
    'red', 'red', 'purple', 'cyan', 'cyan'
]

# 创建主图
fig, ax = plt.subplots(figsize=(10, 6))
for i, model in enumerate(models):
    ax.scatter(gflops[i], map_values[i], s=bubble_size[i], color=colors[i], alpha=0.6, edgecolors="k", label=model if model not in ax.get_legend_handles_labels()[1] else "")

# 标注模型名称
for i, model in enumerate(models):
    ax.annotate(model, (gflops[i], map_values[i] - 1), fontsize=9, ha='center')

# 画对比虚线，并在中间标注差值
comparison_pairs = [
    ("RT-DETRv2", "MSAR-B"),
    ("YOLOV8n", "MSAR-T")
]

for model1, model2 in comparison_pairs:
    i1, i2 = models.index(model1), models.index(model2)
    gflops_diff = gflops[i2] - gflops[i1]
    map_diff = map_values[i2] - map_values[i1]

    if model1 == "RT-DETRv2" and model2 == "MSAR-B":
        ax.plot([gflops[i1], gflops[i2]], [map_values[i1], map_values[i1]], linestyle='dotted', color='blue', alpha=0.7)
        ax.plot([gflops[i2], gflops[i2]], [map_values[i1], map_values[i2]], linestyle='dotted', color='red', alpha=0.7)
        
        ax.text((gflops[i1] + gflops[i2]) / 2, map_values[i1] - 1, f"ΔGFLOPs: {gflops_diff:.1f}", color='blue', fontsize=10, ha='center')
        ax.text(gflops[i2] + 1, (map_values[i1] + map_values[i2]) / 2, f"ΔmAP: {map_diff:.1f}", color='red', fontsize=10, ha='left')

    elif model1 == "YOLOV8n" and model2 == "MSAR-T":
        ax.plot([gflops[i1], gflops[i1]], [map_values[i1], map_values[i2]], linestyle='dotted', color='blue', alpha=0.7)
        ax.plot([gflops[i1], gflops[i2]], [map_values[i2], map_values[i2]], linestyle='dotted', color='red', alpha=0.7)
        
        ax.text(gflops[i1] - 2, (map_values[i1] + map_values[i2]) / 2, f"ΔmAP: {map_diff:.1f}", color='blue', fontsize=10, ha='right')
        ax.text((gflops[i1] + gflops[i2]) / 2, map_values[i2] + 1, f"ΔGFLOPs: {gflops_diff:.1f}", color='red', fontsize=10, ha='center')

# 设定坐标轴标签和标题
ax.set_xlabel("GFLOPs")
ax.set_ylabel("mAP")
ax.set_title("GFLOPs vs. mAP (Bubble Size Represents Model Parameters)")
ax.grid(True, linestyle='--', alpha=0.5)

# 显示图像
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 定义函数将数据坐标转换为 figure 归一化坐标
def data_to_fig_coords(ax, x, y):
    return ax.transData.transform((x, y))  # 将数据坐标转换为屏幕坐标

def fig_to_axes_coords(ax, x, y):
    inv = ax.transAxes.inverted()  # 获取 figure 归一化坐标
    return inv.transform((x, y))  # 转换为 axes 坐标

# 定义模型数据
models = [
    "Faster R-CNN", "Cascade R-CNN", "YOLOV5n", "YOLOV8n", "YOLOV10n", "YOLOV11n", 
    "YOLOV5s", "YOLOV8s", "YOLOV10s", "YOLOV11s", "RT-DETR", "RT-DETRv2", "DINO", "MSAD-T", "MSAD-B"
]

gflops = [91.3, 119.0, 7.1, 8.1, 8.4, 6.4, 23.8, 28.4, 24.4, 21.6, 130.5, 100.5, 119.0, 12.2, 47.8]
map_values = [51.7, 47.9, 67.2, 74.2, 63.4, 68.8, 68.9, 79.9, 73.4, 76.1, 53.8, 64.0, 57.0, 76.3, 81.9]
params = [41.7, 69.4, 2.5, 3.0, 2.7, 2.6, 9.1, 11.1, 8.0, 9.4, 42.7, 36.4, 47.5, 5.5, 20.7]

# 归一化气泡大小
bubble_size = np.array(params) * 10

# 颜色区分不同算法
colors = [
    'blue', 'blue', 'green', 'green', 'green', 'green', 'orange', 'orange', 'orange', 'orange', 
    'red', 'red', 'purple', 'cyan', 'cyan'
]

# 创建主图
fig, ax = plt.subplots(figsize=(10, 6))
for i, model in enumerate(models):
    ax.scatter(gflops[i], map_values[i], s=bubble_size[i], color=colors[i], alpha=0.6, edgecolors="k", label=model if model not in ax.get_legend_handles_labels()[1] else "")

# 标注模型名称
for i, model in enumerate(models):
    ax.annotate(model, (gflops[i], map_values[i] - 1), fontsize=9, ha='center')

# 画对比虚线，并在中间标注差值
comparison_pairs = [
    ("RT-DETRv2", "MSAD-B"),
    ("YOLOV8n", "MSAD-T")
]

for model1, model2 in comparison_pairs:
    i1, i2 = models.index(model1), models.index(model2)
    gflops_diff = gflops[i2] - gflops[i1]
    map_diff = map_values[i2] - map_values[i1]

    if model1 == "RT-DETRv2" and model2 == "MSAD-B":
        ax.plot([gflops[i1], gflops[i2]], [map_values[i1], map_values[i1]], linestyle='dotted', color='blue', alpha=0.7)
        ax.plot([gflops[i2], gflops[i2]], [map_values[i1], map_values[i2]], linestyle='dotted', color='red', alpha=0.7)
        
        ax.text((gflops[i1] + gflops[i2]) / 2, map_values[i1] - 1.2, f"ΔGFLOPs: {np.abs(gflops_diff):.1f}", color='blue', fontsize=10, ha='center')
        ax.text(gflops[i2] + 1, (map_values[i1] + map_values[i2]) / 2, f"ΔmAP: {map_diff:.1f}", color='red', fontsize=10, ha='left')

# 计算放大区域的边界
i1, i2 = models.index("YOLOV8n"), models.index("MSAD-T")
x_min, x_max = min(gflops[i1], gflops[i2]) - 1, max(gflops[i1], gflops[i2]) + 2
y_min, y_max = min(map_values[i1], map_values[i2]) - 1.5, max(map_values[i1], map_values[i2]) + 1

# 在主图上框选出放大区域
ax.plot([x_min, x_max, x_max, x_min, x_min], [y_min, y_min, y_max, y_max, y_min], linestyle="dashed", color="black", alpha=0.4)

# 计算主图框选区域的右上角和右下角（数据坐标系）
ax.plot([x_max, 84.2], [y_max, 79.5], linestyle="dashed", color="black", alpha=0.4)
ax.plot([x_max, 84.2], [y_min, 70], linestyle="dashed", color="black", alpha=0.4)

# 添加放大图，并调整大小
axins = fig.add_axes([0.6, 0.6, 0.2, 0.2])  

# 只绘制YOLOV8n 和 MSAR-T
axins.scatter(gflops[i1], map_values[i1], s=bubble_size[i1], color='green', alpha=0.6, edgecolors="k", label="YOLOV8n")
axins.scatter(gflops[i2], map_values[i2], s=bubble_size[i2], color='cyan', alpha=0.6, edgecolors="k", label="MSAR-T")

# 缩放放大图范围，使其与主图框选区域一致
axins.set_xlim(x_min, x_max)
axins.set_ylim(y_min, y_max)

# 画虚线对比（先竖直后水平）
axins.plot([gflops[i1], gflops[i1]], [map_values[i1], map_values[i2]], linestyle='dotted', color='red', alpha=0.7)
axins.plot([gflops[i1], gflops[i2]], [map_values[i2], map_values[i2]], linestyle='dotted', color='blue', alpha=0.7)

# 在放大图上标注差值
axins.text(gflops[i1] + 2.2, (map_values[i1] + map_values[i2]) / 2 -0.2, f"ΔmAP: {map_diff:.1f}", color='red', fontsize=8, ha='right')
axins.text((gflops[i1] + gflops[i2]) / 2, map_values[i2] + 0.2, f"ΔGFLOPs: {gflops_diff:.1f}", color='blue', fontsize=8, ha='center')

# 放大图内部标题
axins.set_title("YOLOV8n vs. MSAD-T", fontsize=10)
axins.set_xticks([])
axins.set_yticks([])

# 设定坐标轴标签和标题
ax.set_xlabel("GFLOPs")
ax.set_ylabel("mAP(%)")
# ax.set_title("GFLOPs vs. mAP (Bubble Size Represents Model Parameters)")
ax.grid(True, linestyle='--', alpha=0.5)

# 显示图像
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 定义模型数据
models = [
    "Faster R-CNN", "Cascade R-CNN", "YOLOV5n", "YOLOV8n", "YOLOV10n", "YOLOV11n", 
    "YOLOV5s", "YOLOV8s", "YOLOV10s", "YOLOV11s", "RT-DETR", "RT-DETRv2", "DINO", "MSAR-T", "MSAR-B"
]

map_values = [51.7, 47.9, 67.2, 74.2, 63.4, 68.8, 68.9, 79.9, 73.4, 76.1, 53.8, 64.0, 57.0, 76.3, 81.9]
fps_values = [55.6, 43.3, 158.7, 82.0, 238.1, 86.2, 156.3, 70.9, 188.7, 78.7, 59.9, 65.8, 55.6, 125.0, 64.1]
gflops = [91.3, 119.0, 7.1, 8.1, 8.4, 6.4, 23.8, 28.4, 24.4, 21.6, 130.5, 100.5, 119.0, 12.2, 47.8]

# 归一化气泡大小（基于 GFLOPs）
bubble_size = np.array(gflops) * 5

# 颜色区分不同算法
colors = [
    'blue', 'blue', 'green', 'green', 'green', 'green', 'orange', 'orange', 'orange', 'orange', 
    'red', 'red', 'purple', 'cyan', 'cyan'
]

# 创建主图
fig, ax = plt.subplots(figsize=(10, 6))
for i, model in enumerate(models):
    ax.scatter(fps_values[i], map_values[i], s=bubble_size[i], color=colors[i], alpha=0.6, edgecolors="k", label=model if model not in ax.get_legend_handles_labels()[1] else "")

# 标注模型名称
for i, model in enumerate(models):
    ax.annotate(model, (fps_values[i], map_values[i] - 1), fontsize=9, ha='center')

# 设定坐标轴标签和标题
ax.set_xlabel("FPS")
ax.set_ylabel("mAP")
ax.set_title("FPS vs. mAP (Bubble Size Represents GFLOPs)")
ax.grid(True, linestyle='--', alpha=0.5)

# 显示图像
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define model data
models = [
    "Faster R-CNN", "Cascade R-CNN", "YOLOV5n", "YOLOV8n", "YOLOV10n", "YOLOV11n", 
    "YOLOV5s", "YOLOV8s", "YOLOV10s", "YOLOV11s", "RT-DETR", "RT-DETRv2", "DINO", "MSAR-T", "MSAR-B"
]

gflops = [91.3, 119.0, 7.1, 8.1, 8.4, 6.4, 23.8, 28.4, 24.4, 21.6, 130.5, 100.5, 119.0, 12.2, 47.8]
map_values = [51.7, 47.9, 67.2, 74.2, 63.4, 68.8, 68.9, 79.9, 73.4, 76.1, 53.8, 64.0, 57.0, 76.3, 81.9]
params = [41.7, 69.4, 2.5, 3.0, 2.7, 2.6, 9.1, 11.1, 8.0, 9.4, 42.7, 36.4, 47.5, 5.5, 20.7]

# Normalize bubble sizes based on parameters (params M)
bubble_size = np.array(params) * 10

# Create scatter plot
plt.figure(figsize=(10, 6))
scatter = plt.scatter(gflops, map_values, s=bubble_size, alpha=0.6, edgecolors="k")

# Add annotations for each model (position below circles)
for i, model in enumerate(models):
    plt.annotate(model, (gflops[i], map_values[i] - 1), fontsize=9, ha='center')  # Shift text below

# Add comparison arrows with value difference
comparison_pairs = [
    ("RT-DETRv2", "MSAR-B"),  # Comparing RT-DETRv2 and MSAR-B
    ("YOLOV8n", "MSAR-T")     # Comparing YOLOV8n and MSAR-T
]

for model1, model2 in comparison_pairs:
    i1, i2 = models.index(model1), models.index(model2)

    # Compute differences
    gflops_diff = np.abs(gflops[i2] - gflops[i1])
    map_diff = map_values[i2] - map_values[i1]

    # Draw horizontal arrow for GFLOPs difference
    plt.arrow(
        gflops[i1], map_values[i1],
        gflops_diff, 0,
        head_width=0.8, head_length=1, fc='blue', ec='blue', alpha=0.7, linestyle='dotted'
    )
    plt.text(
        (gflops[i1] + gflops[i2]) / 2, map_values[i1] - 1.5,
        f"ΔGFLOPs: {gflops_diff:.1f}", color='blue', fontsize=10, ha='center'
    )

    # Draw vertical arrow for mAP difference
    plt.arrow(
        gflops[i2], map_values[i1],
        0, map_diff,
        head_width=1, head_length=0.8, fc='red', ec='red', alpha=0.7, linestyle='dotted'
    )
    plt.text(
        gflops[i2] + 2, (map_values[i1] + map_values[i2]) / 2,
        f"ΔmAP: {map_diff:.1f}", color='red', fontsize=10, ha='left'
    )

# Labels and title
plt.xlabel("GFLOPs")
plt.ylabel("mAP")
plt.title("GFLOPs vs. mAP (Bubble Size Represents Model Parameters)")
plt.grid(True, linestyle='--', alpha=0.5)

# Show plot
plt.show()


In [ ]:
## 合并json文件
import json

# 需要合并的 JSON 文件
files = ["outputs/sft_data/qwen_article_output.json", "outputs/sft_data/qwen_book_output.json", "outputs/sft_data/qwen_web_output.json"]

# 存储合并后的数据
merged_data = []

for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)  # 读取 JSON
        if isinstance(data, list):
            merged_data.extend(data)  # 合并列表
        else:
            merged_data.append(data)  # 如果是字典，作为列表项添加

# 写入合并后的 JSON 文件
with open("outputs/sft_data/sft_output_all_250220.json", "w", encoding="utf-8") as f:
    json.dump(merged_data, f, ensure_ascii=False, indent=4)

print("JSON 文件合并完成，结果已保存为 merged_output.json")


In [ ]:
## 合并jsonl文件
import json

# 需要合并的 JSONL 文件
# files = ["mateinfo/articals-1112.jsonl", "mateinfo/books-1113.jsonl", "mateinfo/web_deduped-1114.jsonl"]

files = ["outputs/sft_data/final/train_article_data.jsonl", "outputs/sft_data/final/train_book_data.jsonl", "outputs/sft_data/final/train_web_data.jsonl"]

# 目标合并后的文件
output_file = "outputs/sft_data/final/train_data.jsonl"

with open(output_file, "w", encoding="utf-8") as outfile:
    for file in files:
        with open(file, "r", encoding="utf-8") as infile:
            for line in infile:  # 逐行读取
                outfile.write(line)  # 直接写入，不改变格式

print(f"JSONL 文件合并完成，结果已保存为 {output_file}")


In [ ]:
import json

# file_path = "/home/wyp/project/forest/forestllm-main/qwen_article_output.json"
# file_path = "/home/wyp/project/forest/forestllm-main/qwen_book_output.json"
# file_path = "/home/wyp/project/forest/forestllm-main/qwen_web_output.json"
file_path = "sft_output_all_250220.json"  # 59576
# 从文件中读取 JSON 数据
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# 打印 JSON 数据的长度
print(len(json_data))  # 如果是数组，输出数组长度；如果是对象，输出键值对数量

In [ ]:

file_path = "outputs/sft_data/final/train_data.jsonl"  # 替换为你的 JSONL 文件名 31162  4001  516  26645   59576 

with open(file_path, "r", encoding="utf-8") as f:
    line_count = sum(1 for _ in f)

print(f"文件 {file_path} 共有 {line_count} 条数据")


In [ ]:
import json
from collections import Counter, defaultdict

# 文件路径
file_path = "/home/wyp/project/ForestLLM/outputs/article/qwen_article_output.json"
# "/home/wyp/project/ForestLLM/outputs/0113/qwen_web_output.json"


# 查找重复 ID 的函数
def find_duplicate_ids_with_consistency(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)

        # 提取所有的 ID 和对应数据
        id_map = defaultdict(list)
        for entry in data:
            if "id" in entry:
                id_map[entry["id"]].append(entry)

        # 统计每个 ID 的出现次数
        duplicates = {
            id_: entries for id_, entries in id_map.items() if len(entries) > 1
        }
        duplicate_count = len(duplicates)

        # 检查每组重复 ID 数据是否完全一致
        consistency_results = {}
        for id_, entries in duplicates.items():
            # 使用第一条数据作为参考，逐条比对
            reference_entry = json.dumps(entries[0], sort_keys=True)
            all_consistent = all(
                json.dumps(entry, sort_keys=True) == reference_entry
                for entry in entries
            )
            consistency_results[id_] = {
                "count": len(entries),
                "consistent": all_consistent,
            }

        return consistency_results, duplicate_count
    except Exception as e:
        return str(e), 0


# 执行检查
consistency_results, duplicate_count = find_duplicate_ids_with_consistency(file_path)

# 输出结果
print(f"重复的 ID 总数: {duplicate_count}")
print("重复的 ID 检查结果:")
for id_, result in consistency_results.items():
    status = "一致" if result["consistent"] else "不一致"
    print(f"- ID: {id_}, 出现次数: {result['count']}, 数据是否一致: {status}")
print(f"重复的 ID 总数: {duplicate_count}")

In [ ]:
import json

def split_mixed_jsonl(input_file, train_output_file, eval_output_file, batch_size=1000):
    """
    将混合的 JSONL 文件拆分成训练数据 (train) 和 评测数据 (eval)
    """
    train_data = []
    eval_data = []

    # 读取原始 JSONL 文件
    with open(input_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                data = json.loads(line.strip())  # 解析 JSON
                if "messages" in data:  # **训练数据**
                    train_data.append(data)
                elif "history" in data:  # **评测数据**
                    eval_data.append(data)
                else:
                    print(f"⚠️ 未知数据格式，跳过：{data}")  # 遇到无法解析的数据，跳过
            except json.JSONDecodeError:
                print(f"❌ JSON 解析失败，跳过：{line.strip()}")

            # **批量写入，减少 I/O 操作**
            if len(train_data) >= batch_size:
                with open(train_output_file, "a", encoding="utf-8") as train_f:
                    for entry in train_data:
                        train_f.write(json.dumps(entry, ensure_ascii=False) + "\n")
                train_data = []

            if len(eval_data) >= batch_size:
                with open(eval_output_file, "a", encoding="utf-8") as eval_f:
                    for entry in eval_data:
                        eval_f.write(json.dumps(entry, ensure_ascii=False) + "\n")
                eval_data = []

    # **写入剩余数据**
    if train_data:
        with open(train_output_file, "a", encoding="utf-8") as train_f:
            for entry in train_data:
                train_f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    if eval_data:
        with open(eval_output_file, "a", encoding="utf-8") as eval_f:
            for entry in eval_data:
                eval_f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"✅ 训练数据已保存到 {train_output_file}")
    print(f"✅ 评测数据已保存到 {eval_output_file}")

# 示例调用
split_mixed_jsonl("/home/wyp/project/ForestLLM/data_sft/eval_general_qa_readable.jsonl", "/home/wyp/project/ForestLLM/data_sft/train_general_qa.jsonl", "/home/wyp/project/ForestLLM/data_sft/eval_general_qa.jsonl")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap
import os

# 1. 读取嵌入数据
original_embeddings = np.load("outputs/emb_data/bert/llama_embeddings_original_text.npy")  
generated_response_embeddings = np.load("outputs/emb_data/bert/llama_embeddings_generated_response.npy")  
generated_knowledge_embeddings = np.load("outputs/emb_data/bert/llama_embeddings_generated_knowledge.npy")  

# 2. 给数据添加标签
original_labels = ["original"] * len(original_embeddings)
generated_response_labels = ["generated_response"] * len(generated_response_embeddings)
generated_knowledge_labels = ["generated_knowledge"] * len(generated_knowledge_embeddings)

# 3. 合并数据
all_embeddings = np.vstack((original_embeddings, generated_response_embeddings, generated_knowledge_embeddings))
all_labels = np.array(original_labels + generated_response_labels + generated_knowledge_labels)

# **4. 颜色渐进设置**
gradient_palette = sns.blend_palette(["blue", "purple", "red"], n_colors=3, as_cmap=False)

# **5. 形状控制**
style_map = {
    "original": "o",
    "generated_response": "s",
    "generated_knowledge": "D"
}

# 6. 先用 PCA 降到 50 维（加速 t-SNE）
pca_path = "outputs/emb_data/reduced_pca_50.npy"
if os.path.exists(pca_path):
    pca_50_embeddings = np.load(pca_path)
else:
    pca_50 = PCA(n_components=50, random_state=42)
    pca_50_embeddings = pca_50.fit_transform(all_embeddings)
    np.save(pca_path, pca_50_embeddings)

# 7. 用 t-SNE 或 UMAP 进一步降到 2 维
method = "tsne"
low_dim_path = f"outputs/emb_data/reduced_{method}_2d.npy"

if os.path.exists(low_dim_path):
    low_dim_embeddings = np.load(low_dim_path)
else:
    if method == "tsne":
        tsne_reducer = TSNE(n_components=2, perplexity=30, random_state=42)
        low_dim_embeddings = tsne_reducer.fit_transform(pca_50_embeddings)
    elif method == "umap":
        umap_reducer = umap.UMAP(n_components=2, n_neighbors=30, min_dist=0.3, random_state=42)
        low_dim_embeddings = umap_reducer.fit_transform(pca_50_embeddings)
    np.save(low_dim_path, low_dim_embeddings)

# 8. 绘制可视化
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x=low_dim_embeddings[:, 0], 
    y=low_dim_embeddings[:, 1], 
    hue=all_labels, 
    style=all_labels,  # **控制形状**
    palette=gradient_palette,  # **渐变颜色**
    markers=style_map,  # **自定义形状**
    alpha=0.6,  # **透明度**
    s=20  # **点大小**
)

plt.title(f"Embedding Visualization ({method.upper()})")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.legend(title="Data Type", loc="best")

# **保存图片**
img_path = f"outputs/emb_data/embedding_{method}.png"
plt.savefig(img_path, dpi=300)
print(f"📸 Visualization saved as {img_path}")

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import umap.umap_ as umap
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
import os

# 1. 读取嵌入数据
original_embeddings = np.load("outputs/emb_data/bert/llama_embeddings_original_text.npy")  
generated_response_embeddings = np.load("outputs/emb_data/bert/llama_embeddings_generated_response.npy")  
generated_knowledge_embeddings = np.load("outputs/emb_data/bert/llama_embeddings_generated_knowledge.npy")  

# 2. 计算“生成数据”与“原始数据”的相似性
print("🔄 Computing Cosine Similarity with Original Data ...")
response_similarity = cosine_similarity(generated_response_embeddings, original_embeddings)
knowledge_similarity = cosine_similarity(generated_knowledge_embeddings, original_embeddings)

# 3. 获取最高相似度（最接近的训练数据）
max_response_similarity = np.max(response_similarity, axis=1)  # 每个 response 取最高相似度
max_knowledge_similarity = np.max(knowledge_similarity, axis=1)  # 每个 knowledge 取最高相似度

# 4. 计算 Sequence Identity to Training（转换为 0-100%）
sequence_identity = np.concatenate((max_response_similarity, max_knowledge_similarity)) * 100  # 转换为百分比

# 5. UMAP 降维
umap_path = "outputs/emb_data/reduced_umap_2d.npy"
if os.path.exists(umap_path):
    print("✅ Loading UMAP-reduced embeddings from file...")
    low_dim_embeddings = np.load(umap_path)
else:
    print("🔄 Applying UMAP on Similarity Data ...")
    umap_reducer = umap.UMAP(n_components=2, metric="cosine", n_neighbors=15, min_dist=0.3, random_state=42)
    low_dim_embeddings = umap_reducer.fit_transform(sequence_identity.reshape(-1, 1))
    np.save(umap_path, low_dim_embeddings)
    print(f"✅ UMAP embeddings saved to {umap_path}")

# 6. 定义颜色映射（渐变色）
import matplotlib.colors as mcolors

# 颜色范围：从 (100, 125, 125) 深色 → (160, 200, 180) 亮色
cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", [
    (100/255, 125/255, 125/255),  # 低相似度
    (160/255, 200/255, 180/255)   # 高相似度
])

# 7. 绘制可视化
plt.figure(figsize=(10, 6))
sc = plt.scatter(
    low_dim_embeddings[:, 0], 
    low_dim_embeddings[:, 1], 
    c=sequence_identity,  # 用 Sequence Identity 作为颜色
    cmap=cmap,  # 渐变色
    alpha=0.7,  
    s=30,  
    edgecolors="black"
)

# 8. 添加颜色条（Colorbar）
cbar = plt.colorbar(sc)
cbar.set_label("% Sequence Identity to Training")

plt.title("UMAP Visualization Based on Similarity to Original Data")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")

# **保存图片**
img_path = "outputs/emb_data/embedding_umap_similarity.png"
plt.savefig(img_path, dpi=300)
print(f"📸 Visualization saved as {img_path}")

plt.show()


In [ ]:
import json

def is_empty_block(block: dict, keys: list):
    """判断指定 keys 中的字段是否都为空"""
    return all(k in block and (not block[k]) for k in keys)

# 读取原始 JSON 文件
with open('/home/wyp/project/forest/forestllm-main/outputs/0321/欠费中断的第一版qwen_book_output.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 如果顶层是列表，处理每个元素
if isinstance(data, list):
    for item in data:
        if (
            is_empty_block(item.get("question_setter", {}), ["questions"]) and
            is_empty_block(item.get("expert_agent", {}), ["refined_questions"]) and
            is_empty_block(item.get("virtual_teacher", {}), ["processed_results"])
        ):
            item.pop("question_setter", None)
            item.pop("expert_agent", None)
            item.pop("virtual_teacher", None)
else:
    # 单个对象（非列表）情况
    if (
        is_empty_block(data.get("question_setter", {}), ["questions"]) and
        is_empty_block(data.get("expert_agent", {}), ["refined_questions"]) and
        is_empty_block(data.get("virtual_teacher", {}), ["processed_results"])
    ):
        data.pop("question_setter", None)
        data.pop("expert_agent", None)
        data.pop("virtual_teacher", None)

# 写入处理结果
with open('qwen_book_output.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("处理完成：已清理所有完全为空的三元字段。")


In [ ]:
import json

input_path = "outputs/0321/qwen_book_output.json"
output_path = "outputs/0321/qwen_book_output.jsonl"

with open(input_path, "r", encoding="utf-8") as infile:
    data = json.load(infile)  # 加载整个 JSON 数组

with open(output_path, "w", encoding="utf-8") as outfile:
    for item in data:
        outfile.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"已成功将 {len(data)} 条数据从 JSON 转换为 JSONL 格式：{output_path}")
